https://www.kaggle.com/code/robikscube/sentiment-analysis-python-youtube-tutorial/notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk

In [2]:
# Read in data
df = pd.read_csv('cleaned_data.csv')
print(df.shape)
df = df.head(500)
print(df.shape)

(45285, 5)
(500, 5)


In [37]:
df = pd.read_csv('cleaned_data.csv')
#df = df.drop(columns=['Unnamed: 0'])
df

,Unnamed: 0,source,review_description,rating,review_date
0,0,Google Play,thread meta fantastic app staying connected cl...,5,2024-01-03 10:16:17
1,1,Google Play,nothing scammer trust,1,2024-01-01 13:36:11
2,2,Google Play,awesome,5,2023-12-17 22:40:19
3,3,Google Play,nice app,3,2023-12-14 01:49:26
4,4,Google Play,great alternative twitter x,5,2023-12-08 19:32:42
...,...,...,...,...,...
45280,2195,App Store,many false information cant ignore cant hide r...,1,2023-12-13 19:25:04
45281,2196,App Store,app malfunction constantly wont let post anyth...,1,2023-12-13 03:01:16
45282,2197,App Store,thread messaging thread weaving fabric life,5,2023-11-29 13:49:52
45283,2198,App Store,white house need account,5,2023-11-27 18:51:49


In [44]:
example = df['review_description'][421]
print(example)

thread awesome best mean instagram thread god bless lot love


In [45]:
tokens = nltk.word_tokenize(example)
tokens[:10]

['thread',
 'awesome',
 'best',
 'mean',
 'instagram',
 'thread',
 'god',
 'bless',
 'lot',
 'love']

In [25]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DeLL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\DeLL\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\DeLL\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DeLL\AppData\Roaming\nltk_data...


True

In [46]:
tagged = nltk.pos_tag(tokens)
tagged[:10]

[('thread', 'JJ'),
 ('awesome', 'JJ'),
 ('best', 'JJS'),
 ('mean', 'JJ'),
 ('instagram', 'NN'),
 ('thread', 'NN'),
 ('god', 'NN'),
 ('bless', 'NN'),
 ('lot', 'NN'),
 ('love', 'VB')]

In [47]:
entities = nltk.chunk.ne_chunk(tagged)
entities.pprint()

(S
  thread/JJ
  awesome/JJ
  best/JJS
  mean/JJ
  instagram/NN
  thread/NN
  god/NN
  bless/NN
  lot/NN
  love/VB)


In [48]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

sia = SentimentIntensityAnalyzer()

In [27]:
sia.polarity_scores('I am so happy!')

{'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.6468}

In [28]:
sia.polarity_scores('This is the worst thing ever.')

{'neg': 0.451, 'neu': 0.549, 'pos': 0.0, 'compound': -0.6249}

In [49]:
sia.polarity_scores(example)

{'neg': 0.0, 'neu': 0.223, 'pos': 0.777, 'compound': 0.9545}

In [40]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [41]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

D:\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DeLL\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [50]:
# VADER results on example
print(example)
sia.polarity_scores(example)

thread awesome best mean instagram thread god bless lot love


{'neg': 0.0, 'neu': 0.223, 'pos': 0.777, 'compound': 0.9545}

In [51]:
# Run for Roberta Model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores_dict = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scores_dict)

{'roberta_neg': 0.0020494338, 'roberta_neu': 0.012456189, 'roberta_pos': 0.9854944}


In [52]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [55]:
from transformers import pipeline

sent_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

D:\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DeLL\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [66]:
sent_pipeline('somehow mixed topic worst algorithm social medium censor ban freedom speech doesnt exist')

[{'label': 'NEGATIVE', 'score': 0.9994887113571167}]

In [72]:
sent_pipeline('need stop hiding friend feed secondary page add something like hashtags topic following find way quickly change direction main page full random people lot political hate speach side becoming worse twitter longer see political post twitter dont follow political poster')

[{'label': 'NEGATIVE', 'score': 0.9982979893684387}]

In [69]:
sent_pipeline('whats point using app there instagram')

[{'label': 'NEGATIVE', 'score': 0.9916274547576904}]

In [70]:
sent_pipeline('feel significantly cozy low key social medium')

[{'label': 'POSITIVE', 'score': 0.997986912727356}]